In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import time
import statistics

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import mutual_info_classif

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import fbeta_score, make_scorer

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from imblearn.pipeline import Pipeline
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *

from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score
from imblearn.metrics import geometric_mean_score

from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.pipeline import make_pipeline

import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
from scipy import stats

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *
from imblearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler()
from timeit import default_timer as timer

In [2]:
scores={
        'gmean': make_scorer(geometric_mean_score),
        'roc': make_scorer(roc_auc_score),
        'mcc':make_scorer(matthews_corrcoef),
        'accuracy': make_scorer(accuracy_score),
        'sensitivity': make_scorer(sensitivity_score)
        }

In [3]:
data_name= [ 'wisconsin', 
            'vehicle2', 
            'vehicle1',
            'vehicle3', 
            'new-thyroid1',
            'ecoli2',
            'glass6', 
            'yeast',
            'yeast3',
            'ecoli3',
            'yeast-2_vs_4',
            'yeast-0-2-5-6_vs_3-7-8-9',
            'vowel',
            'led7digit-0-2-4-5-6-7-8-9_vs_1',
            'glass2',
            'ecoli-0-1-4-7_vs_5-6',
            'glass4',
            'ecoli4',
            'page-blocks-1-3_vs_4',
            'abalone',
            'yeast-1-4-5-8_vs_7',
            'yeast4',
            'yeast128',
            'winequality-red-8_vs_6',
            'ecoli_013vs26',
            'abalone-17_vs_7-8-9-10',
            'winequality-white-3_vs_7',
            'winequality-red-8_vs_6-7',
            'abalone-19_vs_10-11-12-13',
            'winequality-red-3_vs_5',
            'abalone_20',
            'kddcup-land_vs_satan',
            'abalone19'
            ]
            

In [4]:
len(data_name)

33

In [5]:
gmean = pd.DataFrame(columns=['dataset'])
roc = pd.DataFrame(columns=['dataset'])
mcc = pd.DataFrame(columns=['dataset'])
accuracy = pd.DataFrame(columns=['dataset'])
sensitivity = pd.DataFrame(columns=['dataset'])

In [6]:
count= 1

for ii in data_name:
    data= pd.read_csv('{}.csv'.format(ii))
    x=data.iloc[:,:-1]
    y=data.iloc[:,-1]
    ir=y.value_counts()[0]/y.value_counts()[1];
    ir=round(ir,2)

    gmean= pd.concat([gmean, pd.DataFrame({'dataset':ii, 'Imbalance Ratio':ir},index=[0])])
    roc= pd.concat([roc, pd.DataFrame({'dataset':ii, 'Imbalance Ratio':ir},index=[0])])
    mcc= pd.concat([mcc, pd.DataFrame({'dataset':ii, 'Imbalance Ratio':ir},index=[0])])
    accuracy= pd.concat([accuracy, pd.DataFrame({'dataset':ii, 'Imbalance Ratio':ir},index=[0])])
    sensitivity = pd.concat([sensitivity, pd.DataFrame({'dataset':ii, 'Imbalance Ratio':ir},index=[0])])

    
    count= count+1
    

In [7]:
gmean

,dataset,Imbalance Ratio
0,wisconsin,1.86
0,vehicle2,2.88
0,vehicle1,2.90
0,vehicle3,2.99
0,new-thyroid1,5.11
0,ecoli2,5.44
0,glass6,6.34
0,yeast,8.10
0,yeast3,8.10
0,ecoli3,8.57


In [8]:
# Drop Index inplace
gmean.reset_index(drop=True, inplace=True)
roc.reset_index(drop=True, inplace=True)
mcc.reset_index(drop=True, inplace=True)
accuracy.reset_index(drop=True, inplace=True)
sensitivity.reset_index(drop=True, inplace=True)

### Over-Bagging

In [9]:
count= 1

for ii in data_name:
    data= pd.read_csv('{}.csv'.format(ii))
    x=data.iloc[:,:-1]
    y=data.iloc[:,-1]
    x= sc.fit_transform(x)
    

    Over_Bagging= BalancedBaggingClassifier(random_state=84, sampler = RandomOverSampler(random_state=100))
    steps=[('model',Over_Bagging)]
    pipeline= Pipeline(steps=steps)
    score = cross_validate(pipeline, x,y, cv=StratifiedKFold(5), n_jobs=-1, scoring=scores)
    
    df1=pd.DataFrame(score)
    
    x= df1.mean()*100
    
    gmean.loc[count-1, 'Over-Bagging']= x['test_gmean']
    roc.loc[count-1, 'Over-Bagging']= x['test_roc']
    mcc.loc[count-1, 'Over-Bagging']= x['test_mcc']
    accuracy.loc[count-1, 'Over-Bagging']= x['test_accuracy']
    sensitivity.loc[count-1, 'Over-Bagging']= x['test_sensitivity']
    
    count= count+1
    
    

In [10]:
mcc

,dataset,Imbalance Ratio,Over-Bagging
0,wisconsin,1.86,90.528052
1,vehicle2,2.88,91.893833
2,vehicle1,2.90,38.654234
3,vehicle3,2.99,38.468441
4,new-thyroid1,5.11,89.734770
5,ecoli2,5.44,70.440302
6,glass6,6.34,86.125782
7,yeast,8.10,75.069777
8,yeast3,8.10,74.197739
9,ecoli3,8.57,48.421703


### SMOTE-Bagging

In [15]:
count= 1

for ii in data_name:
    data= pd.read_csv('{}.csv'.format(ii))
    x=data.iloc[:,:-1]
    y=data.iloc[:,-1]
    x= sc.fit_transform(x)
    

    SMOTE_Bagging= BalancedBaggingClassifier(random_state=84, sampler = SMOTE(random_state=100, k_neighbors=3))
    steps=[('model', SMOTE_Bagging)]
    pipeline= Pipeline(steps=steps)
    score = cross_validate(pipeline, x,y, cv=StratifiedKFold(5), n_jobs=-1, scoring=scores)
    
    df1=pd.DataFrame(score)
    
    x= df1.mean()*100
    
    gmean.loc[count-1, 'SMOTE-Bagging']= x['test_gmean']
    roc.loc[count-1, 'SMOTE-Bagging']= x['test_roc']
    mcc.loc[count-1, 'SMOTE-Bagging']= x['test_mcc']
    accuracy.loc[count-1, 'SMOTE-Bagging']= x['test_accuracy']
    sensitivity.loc[count-1, 'SMOTE-Bagging']= x['test_sensitivity']
    
    count= count+1
    
    

D:\apps\anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "D:\apps\anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\apps\anaconda\anaconda\lib\site-packages\imblearn\pipeline.py", line 272, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "D:\apps\anaconda\anaconda\lib\site-packages\imblearn\ensemble\_bagging.py", line 331, in fit
    return super().fit(X, y)
  File "D:\apps\anaconda\an

In [16]:
print(mcc)

                           dataset  Imbalance Ratio  Over-Bagging  \
0                        wisconsin             1.86     90.528052   
1                         vehicle2             2.88     91.893833   
2                         vehicle1             2.90     38.654234   
3                         vehicle3             2.99     38.468441   
4                     new-thyroid1             5.11     89.734770   
5                           ecoli2             5.44     70.440302   
6                           glass6             6.34     86.125782   
7                            yeast             8.10     75.069777   
8                           yeast3             8.10     74.197739   
9                           ecoli3             8.57     48.421703   
10                    yeast-2_vs_4             9.06     71.086574   
11        yeast-0-2-5-6_vs_3-7-8-9             9.13     44.804678   
12                           vowel             9.98     78.580136   
13  led7digit-0-2-4-5-6-7-8-9_vs_1

### NC

In [21]:
rf= RandomForestClassifier(random_state=10)
count= 1

for ii in data_name:
    data= pd.read_csv('{}.csv'.format(ii))
    x=data.iloc[:,:-1]
    y=data.iloc[:,-1]
    x= sc.fit_transform(x)
    

    nc= NeighbourhoodCleaningRule()
    steps=[('sampling', nc),('model',rf)]
    pipeline= Pipeline(steps=steps)
    score = cross_validate(pipeline, x,y, cv=StratifiedKFold(5), n_jobs=-1, scoring=scores)
    
    df1=pd.DataFrame(score)
    
    x= df1.mean()*100
    
    gmean.loc[count-1, 'NC']= x['test_gmean']
    roc.loc[count-1, 'NC']= x['test_roc']
    mcc.loc[count-1, 'NC']= x['test_mcc']
    accuracy.loc[count-1, 'NC']= x['test_accuracy']
    sensitivity.loc[count-1, 'NC']= x['test_sensitivity']
    
    count= count+1
    
    

In [22]:
roc

,dataset,Imbalance Ratio,Over-Bagging,SMOTE-Bagging,NC
0,wisconsin,1.86,94.984963,94.252037,97.173136
1,vehicle2,2.88,94.944738,96.061567,98.050955
2,vehicle1,2.90,67.588285,69.596444,79.091373
3,vehicle3,2.99,66.728461,67.437922,76.111329
4,new-thyroid1,5.11,93.722222,93.722222,95.714286
5,ecoli2,5.44,84.158863,88.249658,88.334188
6,glass6,6.34,92.515015,92.515015,93.618619
7,yeast,8.10,86.307962,87.368568,89.366674
8,yeast3,8.10,85.700758,85.984848,89.346591
9,ecoli3,8.57,72.119048,81.952381,81.285714


### CNN

In [23]:
rf= RandomForestClassifier(random_state=10)
count= 1

for ii in data_name:
    data= pd.read_csv('{}.csv'.format(ii))
    x=data.iloc[:,:-1]
    y=data.iloc[:,-1]
    x= sc.fit_transform(x)
    

    cnn= CondensedNearestNeighbour(random_state=10, n_jobs=-1)
    steps=[('sampling', cnn),('model',rf)]
    pipeline= Pipeline(steps=steps)
    score = cross_validate(pipeline, x,y, cv=StratifiedKFold(5), n_jobs=-1, scoring=scores)
    
    df1=pd.DataFrame(score)
    
    x= df1.mean()*100
    
    gmean.loc[count-1, 'CNN']= x['test_gmean']
    roc.loc[count-1, 'CNN']= x['test_roc']
    mcc.loc[count-1, 'CNN']= x['test_mcc']
    accuracy.loc[count-1, 'CNN']= x['test_accuracy']
    sensitivity.loc[count-1, 'CNN']= x['test_sensitivity']
    
    count= count+1
    
    

In [24]:
mcc

,dataset,Imbalance Ratio,Over-Bagging,SMOTE-Bagging,NC,CNN
0,wisconsin,1.86,90.528052,89.203349,93.737384,92.174009
1,vehicle2,2.88,91.893833,92.270894,95.408171,94.623106
2,vehicle1,2.90,38.654234,41.195754,52.216259,49.618697
3,vehicle3,2.99,38.468441,37.380118,46.714760,42.927883
4,new-thyroid1,5.11,89.734770,89.790218,94.716721,88.927348
5,ecoli2,5.44,70.440302,76.941879,77.760230,74.532264
6,glass6,6.34,86.125782,86.125782,86.536234,82.338630
7,yeast,8.10,75.069777,75.088067,77.426893,76.647412
8,yeast3,8.10,74.197739,72.480884,77.376389,78.142842
9,ecoli3,8.57,48.421703,61.540028,58.144347,54.196504


### over boost

In [25]:
from copy import deepcopy

import numpy as np

from sklearn.base import clone
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble._base import _set_random_states
from sklearn.utils import _safe_indexing

from imblearn.under_sampling.base import BaseUnderSampler
from imblearn.over_sampling.base import BaseOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.utils import Substitution, check_target_type
from imblearn.utils._docstring import _random_state_docstring
from imblearn.utils._validation import _deprecate_positional_args


@Substitution(
    sampling_strategy=BaseOverSampler._sampling_strategy_docstring,
    random_state=_random_state_docstring,
)
class OverBoostClassifier(AdaBoostClassifier):
    """Random under-sampling integrated in the learning of AdaBoost.
    During learning, the problem of class balancing is alleviated by random
    under-sampling the sample at each iteration of the boosting algorithm.
    Read more in the :ref:`User Guide <boosting>`.
    .. versionadded:: 0.4
    Parameters
    ----------
    base_estimator : estimator object, default=None
        The base estimator from which the boosted ensemble is built.
        Support for sample weighting is required, as well as proper
        ``classes_`` and ``n_classes_`` attributes. If ``None``, then
        the base estimator is ``DecisionTreeClassifier(max_depth=1)``.
    n_estimators : int, default=50
        The maximum number of estimators at which boosting is terminated.
        In case of perfect fit, the learning procedure is stopped early.
    learning_rate : float, default=1.0
        Learning rate shrinks the contribution of each classifier by
        ``learning_rate``. There is a trade-off between ``learning_rate`` and
        ``n_estimators``.
    algorithm : {{'SAMME', 'SAMME.R'}}, default='SAMME.R'
        If 'SAMME.R' then use the SAMME.R real boosting algorithm.
        ``base_estimator`` must support calculation of class probabilities.
        If 'SAMME' then use the SAMME discrete boosting algorithm.
        The SAMME.R algorithm typically converges faster than SAMME,
        achieving a lower test error with fewer boosting iterations.
    {sampling_strategy}
    replacement : bool, default=False
        Whether or not to sample randomly with replacement or not.
    {random_state}
    Attributes
    ----------
    base_estimator_ : estimator
        The base estimator from which the ensemble is grown.
    estimators_ : list of classifiers
        The collection of fitted sub-estimators.
    base_sampler_ : :class:`~imblearn.under_sampling.RandomUnderSampler`
        The base sampler used to generate the subsequent samplers.
    samplers_ : list of :class:`~imblearn.under_sampling.RandomUnderSampler`
        The collection of fitted samplers.
    pipelines_ : list of Pipeline
        The collection of fitted pipelines (samplers + trees).
    classes_ : ndarray of shape (n_classes,)
        The classes labels.
    n_classes_ : int
        The number of classes.
    estimator_weights_ : ndarray of shape (n_estimator,)
        Weights for each estimator in the boosted ensemble.
    estimator_errors_ : ndarray of shape (n_estimator,)
        Classification error for each estimator in the boosted
        ensemble.
    feature_importances_ : ndarray of shape (n_features,)
        The feature importances if supported by the ``base_estimator``.
    n_features_in_ : int
        Number of features in the input dataset.
        .. versionadded:: 0.9
    See Also
    --------
    BalancedBaggingClassifier : Bagging classifier for which each base
        estimator is trained on a balanced bootstrap.
    BalancedRandomForestClassifier : Random forest applying random-under
        sampling to balance the different bootstraps.
    EasyEnsembleClassifier : Ensemble of AdaBoost classifier trained on
        balanced bootstraps.
    References
    ----------
    .. [1] Seiffert, C., Khoshgoftaar, T. M., Van Hulse, J., & Napolitano, A.
       "RUSBoost: A hybrid approach to alleviating class imbalance." IEEE
       Transactions on Systems, Man, and Cybernetics-Part A: Systems and Humans
       40.1 (2010): 185-197.
    Examples
    --------
    >>> from imblearn.ensemble import RUSBoostClassifier
    >>> from sklearn.datasets import make_classification
    >>>
    >>> X, y = make_classification(n_samples=1000, n_classes=3,
    ...                            n_informative=4, weights=[0.2, 0.3, 0.5],
    ...                            random_state=0)
    >>> clf = RUSBoostClassifier(random_state=0)
    >>> clf.fit(X, y)  # doctest: +ELLIPSIS
    RUSBoostClassifier(...)
    >>> clf.predict(X)  # doctest: +ELLIPSIS
    array([...])
    """

    @_deprecate_positional_args
    def __init__(
        self,
        base_estimator=None,
        *,
        n_estimators=50,
        learning_rate=1.0,
        algorithm="SAMME.R",
        sampling_strategy="auto",
        replacement=False,
        random_state=None,
    ):
        super().__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            algorithm=algorithm,
            random_state=random_state,
        )
        self.sampling_strategy = sampling_strategy
        self.replacement = replacement

    def fit(self, X, y, sample_weight=None):
        """Build a boosted classifier from the training set (X, y).
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            The training input samples. Sparse matrix can be CSC, CSR, COO,
            DOK, or LIL. DOK and LIL are converted to CSR.
        y : array-like of shape (n_samples,)
            The target values (class labels).
        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, the sample weights are initialized to
            ``1 / n_samples``.
        Returns
        -------
        self : object
            Returns self.
        """
        check_target_type(y)
        self.samplers_ = []
        self.pipelines_ = []
        super().fit(X, y, sample_weight)
        return self

    def _validate_estimator(self):
        """Check the estimator and the n_estimator attribute, set the
        `base_estimator_` attribute."""
        super()._validate_estimator()

        self.base_sampler_ = RandomOverSampler(
            sampling_strategy=self.sampling_strategy,
            
        )

    def _make_sampler_estimator(self, append=True, random_state=None):
        """Make and configure a copy of the `base_estimator_` attribute.
        Warning: This method should be used to properly instantiate new
        sub-estimators.
        """
        estimator = clone(self.base_estimator_)
        estimator.set_params(**{p: getattr(self, p) for p in self.estimator_params})
        sampler = clone(self.base_sampler_)

        if random_state is not None:
            _set_random_states(estimator, random_state)
            _set_random_states(sampler, random_state)

        if append:
            self.estimators_.append(estimator)
            self.samplers_.append(sampler)
            self.pipelines_.append(
                make_pipeline(deepcopy(sampler), deepcopy(estimator))
            )

        return estimator, sampler

    def _boost_real(self, iboost, X, y, sample_weight, random_state):
        """Implement a single boost using the SAMME.R real algorithm."""
        estimator, sampler = self._make_sampler_estimator(random_state=random_state)

        X_res, y_res = sampler.fit_resample(X, y)
        sample_weight_res = _safe_indexing(sample_weight, sampler.sample_indices_)
        estimator.fit(X_res, y_res, sample_weight=sample_weight_res)

        y_predict_proba = estimator.predict_proba(X)

        if iboost == 0:
            self.classes_ = getattr(estimator, "classes_", None)
            self.n_classes_ = len(self.classes_)

        y_predict = self.classes_.take(np.argmax(y_predict_proba, axis=1), axis=0)

        # Instances incorrectly classified
        incorrect = y_predict != y

        # Error fraction
        estimator_error = np.mean(np.average(incorrect, weights=sample_weight, axis=0))

        # Stop if classification is perfect
        if estimator_error <= 0:
            return sample_weight, 1.0, 0.0

        # Construct y coding as described in Zhu et al [2]:
        #
        #    y_k = 1 if c == k else -1 / (K - 1)
        #
        # where K == n_classes_ and c, k in [0, K) are indices along the second
        # axis of the y coding with c being the index corresponding to the true
        # class label.
        n_classes = self.n_classes_
        classes = self.classes_
        y_codes = np.array([-1.0 / (n_classes - 1), 1.0])
        y_coding = y_codes.take(classes == y[:, np.newaxis])

        # Displace zero probabilities so the log is defined.
        # Also fix negative elements which may occur with
        # negative sample weights.
        proba = y_predict_proba  # alias for readability
        np.clip(proba, np.finfo(proba.dtype).eps, None, out=proba)

        # Boost weight using multi-class AdaBoost SAMME.R alg
        estimator_weight = (
            -1.0
            * self.learning_rate
            * ((n_classes - 1.0) / n_classes)
            * (y_coding * np.log(y_predict_proba)).sum(axis=1)
        )

        # Only boost the weights if it will fit again
        if not iboost == self.n_estimators - 1:
            # Only boost positive weights
            sample_weight *= np.exp(
                estimator_weight * ((sample_weight > 0) | (estimator_weight < 0))
            )

        return sample_weight, 1.0, estimator_error

    def _boost_discrete(self, iboost, X, y, sample_weight, random_state):
        """Implement a single boost using the SAMME discrete algorithm."""
        estimator, sampler = self._make_sampler_estimator(random_state=random_state)

        X_res, y_res = sampler.fit_resample(X, y)
        sample_weight_res = _safe_indexing(sample_weight, sampler.sample_indices_)
        estimator.fit(X_res, y_res, sample_weight=sample_weight_res)

        y_predict = estimator.predict(X)

        if iboost == 0:
            self.classes_ = getattr(estimator, "classes_", None)
            self.n_classes_ = len(self.classes_)

        # Instances incorrectly classified
        incorrect = y_predict != y

        # Error fraction
        estimator_error = np.mean(np.average(incorrect, weights=sample_weight, axis=0))

        # Stop if classification is perfect
        if estimator_error <= 0:
            return sample_weight, 1.0, 0.0

        n_classes = self.n_classes_

        # Stop if the error is at least as bad as random guessing
        if estimator_error >= 1.0 - (1.0 / n_classes):
            self.estimators_.pop(-1)
            self.samplers_.pop(-1)
            self.pipelines_.pop(-1)
            if len(self.estimators_) == 0:
                raise ValueError(
                    "BaseClassifier in AdaBoostClassifier "
                    "ensemble is worse than random, ensemble "
                    "can not be fit."
                )
            return None, None, None

        # Boost weight using multi-class AdaBoost SAMME alg
        estimator_weight = self.learning_rate * (
            np.log((1.0 - estimator_error) / estimator_error) + np.log(n_classes - 1.0)
        )

        # Only boost the weights if I will fit again
        if not iboost == self.n_estimators - 1:
            # Only boost positive weights
            sample_weight *= np.exp(estimator_weight * incorrect * (sample_weight > 0))

        return sample_weight, estimator_weight, estimator_error

In [26]:
count= 1

for ii in data_name:
    data= pd.read_csv('{}.csv'.format(ii))
    x=data.iloc[:,:-1]
    y=data.iloc[:,-1]
    x= sc.fit_transform(x)
    

    over_boost= OverBoostClassifier(random_state=84)
    steps=[('model',over_boost)]
    pipeline= Pipeline(steps=steps)
    score = cross_validate(pipeline, x,y, cv=StratifiedKFold(5), n_jobs=-1, scoring=scores)
    
    df1=pd.DataFrame(score)
    
    x= df1.mean()*100
    
    gmean.loc[count-1, 'Over_Boost']= x['test_gmean']
    roc.loc[count-1, 'Over_Boost']= x['test_roc']
    mcc.loc[count-1, 'Over_Boost']= x['test_mcc']
    accuracy.loc[count-1, 'Over_Boost']= x['test_accuracy']
    sensitivity.loc[count-1, 'Over_Boost']= x['test_sensitivity']
    
    count= count+1
    
    

In [27]:
mcc

,dataset,Imbalance Ratio,Over-Bagging,SMOTE-Bagging,NC,CNN,Over_Boost
0,wisconsin,1.86,90.528052,89.203349,93.737384,92.174009,88.458366
1,vehicle2,2.88,91.893833,92.270894,95.408171,94.623106,94.241222
2,vehicle1,2.90,38.654234,41.195754,52.216259,49.618697,41.996465
3,vehicle3,2.99,38.468441,37.380118,46.714760,42.927883,43.568631
4,new-thyroid1,5.11,89.734770,89.790218,94.716721,88.927348,94.968505
5,ecoli2,5.44,70.440302,76.941879,77.760230,74.532264,69.667825
6,glass6,6.34,86.125782,86.125782,86.536234,82.338630,88.527228
7,yeast,8.10,75.069777,75.088067,77.426893,76.647412,73.219436
8,yeast3,8.10,74.197739,72.480884,77.376389,78.142842,74.653906
9,ecoli3,8.57,48.421703,61.540028,58.144347,54.196504,49.203359


In [28]:
with pd.ExcelWriter('ibrf_other_models_results.xlsx') as writer:  
         gmean.to_excel(writer, sheet_name='gmean')
         mcc.to_excel(writer, sheet_name='mcc')
         roc.to_excel(writer, sheet_name='roc')
         accuracy.to_excel(writer, sheet_name='accuracy')
         sensitivity.to_excel(writer, sheet_name='sensitivity')
        